In [12]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [13]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [14]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>Atrial Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [4]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/hebbian_unsup/').glob('unet3d_swta_t')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet3D-SWTA-T,0,1,100,34.85,21.10
1,H-UNet3D-SWTA-T,0,5,100,21.15,11.83
2,H-UNet3D-SWTA-T,0,10,100,22.57,12.72
3,H-UNet3D-SWTA-T,0,20,100,25.66,14.72
4,H-UNet3D-SWTA-T,0,50,100,27.22,15.76
5,H-UNet3D-SWTA-T,0,75,100,20.24,11.26
6,H-UNet3D-SWTA-T,0,100,100,20.21,11.24


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                  Mean CI 0.9%    Mean CI 0.9%
Model           Inv Temp Regime                               
H-UNet3D-SWTA-T 1        100     34.85     NaN   21.10     NaN
                5        100     21.15     NaN   11.83     NaN
                10       100     22.57     NaN   12.72     NaN
                20       100     25.66     NaN   14.72     NaN
                50       100     27.22     NaN   15.76     NaN
                75       100     20.24     NaN   11.26     NaN
                100      100     20.21     NaN   11.24     NaN

In [28]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_unet3d_swta_t')),
    'H-EM-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_em_unet3d_swta_t')),
}

In [29]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [30]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-EM-SWTA-T,0,1,1,26.37,15.19
1,H-EM-SWTA-T,0,1,2,32.43,19.35
2,H-EM-SWTA-T,0,1,5,39.55,24.65
3,H-EM-SWTA-T,0,1,10,36.81,22.56
4,H-EM-SWTA-T,0,1,20,41.60,26.26
...,...,...,...,...,...,...
165,H-EM-SWTA-T,4,75,20,88.47,79.32
166,H-EM-SWTA-T,4,100,1,53.94,36.93
167,H-EM-SWTA-T,4,100,2,70.02,53.86
168,H-EM-SWTA-T,4,100,5,79.86,66.47


Dice           Jaccard          
                               Mean   CI 0.9%    Mean   CI 0.9%
Model       Inv Temp Regime                                    
H-EM-SWTA-T 1        1       31.522  6.702839  18.916  4.625562
                     2       36.106  3.198399  22.082  2.383206
                     5       37.154  3.715273  22.886  2.740826
                     10      36.018  1.324938  21.974  0.978749
                     20      38.702  2.600108  24.032  1.985777
            5        1       21.206  1.626683  11.870  1.020850
                     2       25.790  7.431516  15.052  5.260010
                     5       24.114  1.554859  13.722  1.016842
                     10      38.154  5.980987  23.754  4.429428
                     20      40.952  3.057537  25.802  2.399671
            10       1       24.906  1.355112  14.232  0.885215
                     2       27.782  6.327434  16.316  4.533754
                     5       29.630  6.656053  17.600  4.894583
                     10      34.324  9.354372  21.156  6.999706
                     20      49.596  1.522283  32.992  1.363948
            20       1       51.158  6.076376  34.622  5.576430
                     2       58.844  3.431426  41.778  3.453989
                     5       70.776  6.924715  55.256  8.237693
                     10      76.500  5.848187  62.328  7.391544
                     20      83.684  2.819360  72.046  4.046066
            50       1       67.048  5.836121  50.758  6.780451
                     2       78.434  3.659524  64.674  4.778509
                     5       82.622  2.138976  70.452  3.100784
                     10      86.338  1.758322  76.004  2.668020
                     20      88.424  0.627143  79.256  1.009849
            75       1       63.654  4.599601  46.866  4.931037
                     2       76.814  4.880537  62.626  6.209790
                     5       81.296  4.014383  68.692  5.578526
                     10      85.540  2.516063  74.820  3.717085
                     20      88.702  0.465481  79.694  0.752722
            100      1       62.130  5.315104  45.302  5.564914
                     2       75.432  4.352241  60.770  5.670882
                     5       82.340  2.220210  70.046  3.224551
                     20      88.182  0.464034  78.866  0.739836

<h2>Evaluation - Weight init comparison</h2>

<p>Evaluate weight init methods (only for baseline)</p>

In [ ]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'Kaiming-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    'Xavier-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('xavier_unet3d')),
    'Orthogonal-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('orthogonal_unet3d')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [12]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    #'VNet': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_vnet')),
    #'EM-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_em_unet3d')),
    #'CPS-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_cps_unet3d')),
    #'UAMT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_uamt_unet3d')),
    #'URPC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_urpc_unet3d')),
    'CCT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_cct_unet3d')),
    #'DTC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_dtc_unet3d')),
}

In [13]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,CCT-UNet3D,0,1,1,64.70,47.82
1,CCT-UNet3D,0,1,2,78.48,64.59
2,CCT-UNet3D,0,1,5,85.54,74.73
3,CCT-UNet3D,0,1,10,87.20,77.29
4,CCT-UNet3D,0,1,20,90.93,83.36
5,CCT-UNet3D,1,1,1,56.06,38.95
6,CCT-UNet3D,1,1,2,74.89,59.86
7,CCT-UNet3D,1,1,5,84.89,73.75
8,CCT-UNet3D,1,1,10,89.04,80.25
9,CCT-UNet3D,1,1,20,91.02,83.52


Dice           Jaccard          
                              Mean   CI 0.9%    Mean   CI 0.9%
Model      Inv Temp Regime                                    
CCT-UNet3D 1        1       62.932  6.055446  46.226  6.404382
                    2       78.636  2.741297  64.886  3.720798
                    5       85.110  0.497776  74.080  0.753239
                    10      88.802  1.070474  79.872  1.739779
                    20      90.742  0.522590  83.052  0.867227